# ANTGO 开发流水线
## ANTGO 环境安装

In [ ]:
# 安装antgo库
!pip3 install antgo@git+https://github.com/jianzfb/antgo.git

In [ ]:
# 安装docker环境
!pip3 install udocker
!udocker --allow-root install

In [ ]:
# 安装android扩展依赖
!cd / && wget https://dl.google.com/android/repository/android-ndk-r20b-linux-x86_64.zip && unzip android-ndk-r20b-linux-x86_64.zip

In [ ]:
# 安装C++扩展依赖
!antgo install opencv
!antgo install eigen
!antgo install grpc
!antgo install ffmpeg
!antgo install eagleeye

In [ ]:
# 挂载google drive并设置工作目录
from google.colab import drive
import os
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/workspace', exist_ok=True)
# 如果已经创建了项目，则直接将项目目录设置为工作目录
os.chdir("/content/drive/MyDrive/workspace/{{project}}")

## ANTGO 环境配置（远程GPU训练部署集群）

In [ ]:
# 配置远程训练平台
# ssh 模式
# step 1: 生成ssh提交配置模板，执行后在当前目录生成ssh-submit-config.yaml
!antgo submitter template
# step 2: 填写模板
'''
username,ip 登录用户名和ip地址。提交后自动创建ssh-key，并设置免密登录。期间会
要求填写密码操作（仅需一次）。
config:
  username: ''
  password: ''
  ip: ''

script: ''
image: ''
'''
# step 3: 更新配置
!antgo submitter update --config=./ssh-submit-config.yaml

# step 4: 查看远程机器
!antgo ls

# k8s 模式(即将支持)

## 模型研发第一步：神经网络模型

### 创建模板项目
#### 创建分类项目
```
!cd /content/drive/MyDrive/workspace/cifar10
!os.chdir("/content/drive/MyDrive/workspace/cifar10")
!antgo create mvp --name=cifar10
```
#### 创建目标检测项目
```
!cd /content/drive/MyDrive/workspace/coco
!os.chdir("/content/drive/MyDrive/workspace/coco")
!antgo create mvp --name=coco
```
#### 创建关键点检测项目
```
!cd /content/drive/MyDrive/workspace/lsp
!os.chdir("/content/drive/MyDrive/workspace/lsp")
!antgo create mvp --name=lsp
```
#### 创建语义分割项目
```
!cd /content/drive/MyDrive/workspace/pascal_voc
!os.chdir("/content/drive/MyDrive/workspace/pascal_voc")
!antgo create mvp --name=pascal_voc
```

### 模型训练脚本

In [ ]:
# 深度神经网络模型训练
!antgo train --exp=experiment_name --config=experiment_config --gpu-id=0 --no-validate --version=master --ip=remote_machine_ip

### 模型测试脚本

In [ ]:
# 深度神经网络模型测试
!antgo export --exp=experiment_name --config=experiment_config --checkpoint=checkpoint_path

### 模型导出脚本

In [ ]:
# 深度神经网络模式导出
!antgo export --exp=experiment_name --config=experiment_config

## 模型研发第二部：管线服务

### 管线创建和运行
> 常用来快速验证模型或功能代码

In [ ]:
import os
import cv2
from antgo.pipeline import *
# 服务管线创建并执行
def debug_show(depth_pred):
    depth_pred = depth_pred[0,0]
    depth_pred = depth_pred / np.max(depth_pred)
    depth_pred = np.clip(depth_pred * 255,0,255)
    cv2.imwrite('./depth_pred.png', depth_pred.astype(np.uint8))

# 下载图片和模型
if not os.path.exists('./office.jpeg'):
  !wget http://file.vibstring.com/office.jpeg
if not os.path.exists('./depth_est.onnx'):
  !wget http://file.vibstring.com/depth_est.onnx

# 管线搭建和运行
# 常用数据源imread_dc, video_dc, glob
# imread_dc 图像读取
# video_dc  视频读取
# glob      文件读取 
imread_dc['image']('./office.jpeg'). \
    resize_op['image', 'resized_image'](out_size=(640,480)). \
    inference_onnx_op['resized_image', ('depth_pred')](
        onnx_path='./depth_est.onnx', 
        mean=[0.485*255, 0.456*255, 0.406*255],
        std=[0.229*255, 0.224*255, 0.225*255]
    ). \
    runas_op['depth_pred', 'o'](func=debug_show). \
    run()

### 管线SDK构建打包

#### 管线SDK编译
> 需要设置目标平台 android/arm64-v8a, linux/x86-64, linux/arm64

In [ ]:
# android/arm64-v8a
# 模型引擎可选: snpe, tnn, rknn
# linux/x86-64
# 模型引擎可选: tensorrt
# linux/arm64
# 模型引擎可选: tensorrt, rknn
placeholder['image'](np.zeros((480,640,3), dtype=np.uint8)). \
    resize_op['image', 'resized_image'](out_size=(640,480)). \
    inference_onnx_op['resized_image', ('depth_pred')](
        onnx_path='./depth_est.onnx', 
        mean=[0.485*255, 0.456*255, 0.406*255],
        std=[0.229*255, 0.224*255, 0.225*255],
        engine='tensorrt'
    ). \
    build(
        platform='linux/x86-64',
        project_config={
            'input': [
                ('image', 'EAGLEEYE_SIGNAL_RGB_IMAGE'),
            ],
            'output': [
                ('depth_pred', 'EAGLEEYE_SIGNAL_TENSOR')
            ],
            'name': 'metricdepth',
            'git': ''
        }
    )

#### 管线SDK打包
> 支持android/sdk, linux/sdk

> 需要根据编译时指定的目标来进行设置

In [ ]:
# 将相关文件打包到package文件夹下
!antgo package --name=metricdepth --mode=linux/sdk

### 管线WEB服务打包

#### 创建server文件夹

In [ ]:
!mkdir server
os.chdir("./server")

#### 编写服务管线

In [ ]:
%%writefile server.py
from antgo.pipeline import *
import cv2
import os
import numpy as np
# 创建web云端部署
def convert_depth_to_show(depth_pred):
    depth_pred = depth_pred[0,0]
    depth_pred = depth_pred / np.max(depth_pred)
    depth_pred = np.clip(depth_pred * 255,0,255)
    cv2.imwrite('./depth_pred.png', depth_pred.astype(np.uint8))

with web['image'](name='demo') as handler:
    app = handler.resize_op['image', 'resized_image'](out_size=(640,480)). \
    inference_onnx_op['resized_image', ('depth_pred')](
        onnx_path='../depth_est.onnx',
        mean=[0.485*255, 0.456*255, 0.406*255],
        std=[0.229*255, 0.224*255, 0.225*255]
    ). \
    runas_op['depth_pred', 'depth_image'](func=convert_depth_to_show). \
    demo(
        title="深度估计",
        description="深度估计DEMO",
		input=[
			{'data': 'image', 'type': 'image'},
		],
		output=[
			{'data': 'depth_image', 'type': 'image'},
		]
    )

#### 启动web服务

In [ ]:
# https://dashboard.ngrok.com/get-started/your-authtoken
!antgo web --main=server:app --port=8080 --ngrok --authtoken=Your Ngrok token

#### 打包服务
> 打包后生成depth镜像

In [ ]:
!antgo package --name=metricdepth --main=server:app --port=8080

#### 部署服务
> 将创建的depth镜像推送到部署机器并部署（需要docker支持）

In [ ]:
!antgo deploy --ip=remote_machine_ip --port=80

### 管线私有化打包
> grpc服务

#### 编写服务管线

In [ ]:
placeholder['image'](np.zeros((480,640,3), dtype=np.uint8)). \
    resize_op['image', 'resized_image'](out_size=(640,480)). \
    inference_onnx_op['resized_image', ('depth_pred')](
        onnx_path='./depth_est.onnx', 
        mean=[0.485*255, 0.456*255, 0.406*255],
        std=[0.229*255, 0.224*255, 0.225*255],
        engine='tensorrt'
    ). \
    build(
        platform='linux/x86-64',
        project_config={
            'input': [
                ('image', 'EAGLEEYE_SIGNAL_RGB_IMAGE'),
            ],
            'output': [
                ('depth_pred', 'EAGLEEYE_SIGNAL_TENSOR')
            ],
            'name': 'metricdepth',
            'git': '',
            'mode': 'server'
        }
    )

#### 打包私有化镜像

In [ ]:
!antgo package --name=metricdepth --mode=grpc --port=8080

#### 部署服务

In [ ]:
!antgo deploy --ip=remote_machine_ip --port=80